In [1]:
import pandas as pd
from scipy import stats

In [2]:
df = pd.read_csv('Results_TD_CP.csv')

metrics = ['dti_fa', 'md', 'ad', 'rd']

In [5]:
comparisons = [
    # Wilcoxon
    ('CSC_LA', 'CSC_MA', 'wilcoxon'),
    ('PV_LA',  'PV_MA',  'wilcoxon'),
    
    # Mann-Whitney
    ('CSC_LA', 'TD_D',   'mannwhitney'),
    ('PV_LA',  'TD_D',   'mannwhitney'),
    ('PV_LA',  'CSC_LA', 'mannwhitney'),
    ('CSC_MA', 'TD_ND',  'mannwhitney'),
    ('PV_MA',  'TD_ND',  'mannwhitney'),
    ('PV_MA',  'CSC_MA', 'mannwhitney')
]

# Bonferroni correction for 8 tests
threshold = 0.05 / 8  

In [9]:
print(f"Threshold p < {threshold:.5f}")
print("-" * 65)
print(f"{'Metric':<8} | {'Comparison':<16} | {'Test':<10} | {'p-value':<10} | {'Sig?'}")
print("-" * 65)

for metric in metrics:
    subset = df[df['Tracts Name'] == metric]
    
    print(f"{metric}")
    
    for g1, g2, test_type in comparisons:
        # For Wilcoxon, we must drop NaN rows together (pairwise deletion)
        if test_type == 'wilcoxon':
            paired_data = subset[[g1, g2]].dropna()
            data1 = paired_data[g1]
            data2 = paired_data[g2]
            
            if len(data1) < 3: 
                print(f"{'':<8} | {g1} vs {g2} | Wilcoxon   | N/A        | Not enough data")
                continue
                
            stat, p_val = stats.wilcoxon(data1, data2)

        # For Mann-Whitney, we drop NaNs individually
        else:
            data1 = subset[g1].dropna()
            data2 = subset[g2].dropna()
            
            if len(data1) == 0 or len(data2) == 0:
                continue
                
            stat, p_val = stats.mannwhitneyu(data1, data2, alternative='two-sided')
            
        # Checking Significance
        sig = "Yes" if p_val < threshold else "No"
        
        print(f"{'':<8} {g1} vs {g2:<6} | {test_type:<10} | {p_val:.2e}   | {sig}")
    print("")

Threshold p < 0.00625
-----------------------------------------------------------------
Metric   | Comparison       | Test       | p-value    | Sig?
-----------------------------------------------------------------
dti_fa
         CSC_LA vs CSC_MA | wilcoxon   | 1.25e-01   | No
         PV_LA vs PV_MA  | wilcoxon   | 2.34e-02   | No
         CSC_LA vs TD_D   | mannwhitney | 4.85e-01   | No
         PV_LA vs TD_D   | mannwhitney | 9.09e-01   | No
         PV_LA vs CSC_LA | mannwhitney | 6.22e-01   | No
         CSC_MA vs TD_ND  | mannwhitney | 2.97e-04   | Yes
         PV_MA vs TD_ND  | mannwhitney | 2.06e-03   | Yes
         PV_MA vs CSC_MA | mannwhitney | 1.71e-01   | No

md
         CSC_LA vs CSC_MA | wilcoxon   | 1.25e-01   | No
         PV_LA vs PV_MA  | wilcoxon   | 7.81e-03   | No
         CSC_LA vs TD_D   | mannwhitney | 9.76e-01   | No
         PV_LA vs TD_D   | mannwhitney | 5.97e-01   | No
         PV_LA vs CSC_LA | mannwhitney | 4.35e-01   | No
         CSC_MA vs TD_ND  | ma